In [3]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import pandas as pd 
from scipy.optimize import minimize
from plotting import plot

In [4]:
Cd = 0.8
S_ref = 10
thrust = 2100000.0
g0 = 9.80665
Isp = 350
params = [Cd,S_ref, thrust, g0, Isp ]

In [5]:
@numba.njit
def dynamics(x, u, params):
    Cd = params[0]
    S_ref = params[1]
    thrust = params[2]
    g0 = params[3]
    Isp = params[4]
    Cda = Cd * S_ref 
    cos_theta  = np.cos(u)
    sin_theta  = np.sin(u)
    rho = 1.225 * np.exp( x[:,1]  /8.44e3)

    dx = np.zeros_like(x)
    dx[:,0] = x[:,3]
    dx[:,1] = x[:,4]
    v_mag_sq = dx[:,0] **2 + dx[:,1]**2
    fd_ = 0.5 * rho * Cda * np.sqrt(v_mag_sq) 
    dx[:,3] =  (thrust * cos_theta - fd_ * dx[:,0]**2 ) / x[:,5] 
    dx[:,4] =  (thrust * sin_theta - fd_ * dx[:,1]**2 )  / x[:,5] - g0
    dx[:,5] =  thrust / g0 / Isp
    return dx

@numba.njit
def objective(N, dt):
    return N*dt

# Define the dynamics defects
@numba.njit
def state_defects(decision_variables, args):
    N = args[0] 
    states_dim = args[1] 
    dt = args[2]
    # Index [0 N )in the decision_variables vector contains the control inputs 
    u = decision_variables[:N]
    # Index [N -1] in the decision_variables vector contains the state  
    x = decision_variables[N:].reshape((N+1, states_dim))
    # Calculate the dynamics
    x_dot = dynamics(x, u)
    # Calculate the approximation of integral using trapezoidal quadrature
    integral = ((x_dot[:-1] + x_dot[1:])) / 2 * dt
    # Calculate the state defects
    defects = x[1:] - x[:-1]  - integral

    return defects.transpose().flatten()

In [6]:
# Define the initial and final states 
mue = 3.986004418e14
v_cir = np.sqrt(mue /6_378_200 )
x0 = [0.0, 0.0, 0.0, 0.0, 117_000]  # initial position and velocity mass
xf = [None, 185_000, v_cir, 0.0, None]  # final position and velocity 
states_dim = 5
N = 50 # Number of collocation points, 

# Define the number of time steps

# Define the constants


#bounds of pitch angle
u_b = np.pi /2 
l_b = -np.pi /2 

In [7]:
final_mass_guess = 0.1 *  117_000 
final_time_guess = 117_000* 0.9 / (thrust / g0 / Isp)


In [8]:
# phase.set_time_val(initial=0.0, duration=150.0)
# phase.set_state_val('x', [0, 1.15E5])
# phase.set_state_val('y', [0, 1.85E5])
# phase.set_state_val('vy', [1.0E-6, 0])
# phase.set_state_val('m', [117000, 1163])
# phase.set_control_val('theta', [1.5, -0.76])
# phase.set_parameter_val('thrust', 2.1, units='MN')

In [9]:
# Initial guess for control inputs
u_init = np.zeros(N)
u_init = np.linspace(1.5, -0.76, N) # Decision variable
t_init  = np.linspace(0, final_time_guess, N+1) 
dt = t_init[1] - t_init[0]  # time step
params =(N,states_dim,dt)

# Initial guess for states
x_init = np.zeros((N+1, states_dim))
x_init[:, 0] = np.linspace(x0[0], 1.115e5, N+1) 
x_init[:, 1] = np.linspace(0, 1.85e5, N+1)
x_init[:, 2] = np.linspace(0, v_cir, N+1)
x_init[:, 3] = np.linspace(0, 0.0, N+1)
x_init[:, 4] = np.linspace(117_000, 0.1 *  117_000, N+1)

# Concatenate control inputs and states into a single decision variable
initial_guess = np.concatenate([u_init, x_init.flatten()])


In [ ]:
plot( t_init, [x_init[:, 0], x_init[:, 1]])

In [ ]:
## Plot initial Guesses 
plot( t_init[:-1], u_init)

In [ ]:

# Define the optimization problem
def problem(decision_variables, args):
    N = args[0] 
    dt = decision_variables[N+1]

    obj_value = objective(N, dt)
    
    return obj_value

# Define the bounds for the decision variables
bounds = [(l_b, l_b)] * N 
num_state_bounds = states_dim*(N+1)

state_bounds = [(0, None)] * num_state_bounds

# for i in range(0,N+1):
#     state_bounds[states_dim*i] = (0,None)


bounds = bounds + state_bounds    
#Enforcing Bound constraint on initial and final states
bounds[N] = (0.0,1.85e5) # x 
bounds[N+1] = (1.85e5,1.85e5) # y
bounds[N+2] = (0.0, 0.0) # v_x 
bounds[N +3] = (v_cir, v_cir) # v_y 
bounds[N +4] = (0, 10000) # mass

# Final Bounds
bounds[N+ num_state_bounds - states_dim + 0] = (1.0,1.0)
bounds[N+ num_state_bounds - states_dim + 1] = (np.pi,np.pi)
bounds[N+ num_state_bounds - states_dim + 2] = (0.0,0.0)
bounds[N+ num_state_bounds - states_dim + 3] = (0.0,0.0)
bounds[N+ num_state_bounds - states_dim + 3] = (0.0,0.0)

arguments = (params,)
# Define the constraints
constraints = [{'type': 'eq', 'fun': state_defects, 'args':arguments },]
# Solve the optimization problem

result = minimize(problem, initial_guess, method='SLSQP', bounds=bounds, args=arguments, constraints=constraints)

In [ ]:
# Extract the optimal control inputs and states
u_opt = result.x[:N]

x_opt = result.x[N:].reshape((N+1, states_dim))